In [46]:
from dotenv import load_dotenv, find_dotenv
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel, VisitWebpageTool, WikipediaSearchTool, FinalAnswerTool, PythonInterpreterTool
# import nest_asyncio

# nest_asyncio.apply()

_ = load_dotenv(find_dotenv())

In [2]:
all_inference_models = !curl -s https://huggingface.co/api/models | jq ".[].id"

In [2]:
from huggingface_hub import model_info


In [3]:

info = model_info("Qwen/Qwen2.5-VL-72B-Instruct", expand="inference")
info.inference


'warm'

In [22]:
vlm_model = InferenceClientModel(model_id="Qwen/Qwen2.5-VL-72B-Instruct",bill_to="VitalNest",
                                 timeout=300,temperature=.1, max_tokens=10000)

In [1]:
from PIL import Image
from io import BytesIO
import base64
from pathlib import Path

In [2]:
def encode_image_to_base64(image_path):
    """
    Encodes an image file to a base64 string.

    Args:
        image_path (str): Path to the image file.

    Returns:
        str: Base64 encoded image string.
    """
    """Encodes a PIL image to a base64 string."""
    image = Image.open(image_path)
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


In [36]:
# MAXIMUM of 4 images can be sent
img_paths = [
    Path(
        "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_42.png"
    ),
    Path(
        "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_12.png"
    ),
    Path(
        "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_1.png"
    ),
    # Path(
    #     "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_43.png"
    # ),
    # Path(
    #     "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_44.png"
    # ),
]

In [37]:
# grouped_images = [Image.open(fp=pth) for pth in img_paths]
grouped_images = [encode_image_to_base64(pth) for pth in img_paths]

In [38]:
text_query = """what critical illnesses are covered under optima restore?
If the answer isn't in these documents, say you don't know."""
chat_template = [
    {
"role":"system", "content":"""You find answers from the relevant documents. Answer only 
from these documents. If answer isn't available return 'Question cannot be answered based
on the documents provided.' """
    },
    {
        "role": "user",
        "content": [
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            for image in grouped_images
        ]
        + [{"type": "text", "text": text_query}],
    }
]


In [39]:
print(vlm_model.generate(messages=chat_template).content)

The critical illnesses covered under Optima Restore include Cancer, Open Chest CABG, First Heart Attack, Kidney Failure, Major Organ/Bone Marrow Transplant, Multiple Sclerosis, Permanent Paralysis of Limbs, and Stroke.


In [40]:
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="hyperbolic",
    bill_to="VitalNest",
)


In [41]:
completion = client.chat.completions.create(
    model="Qwen/Qwen2.5-VL-72B-Instruct",
    messages=chat_template,
    temperature=.1,
    max_tokens=10_000
)

print(completion.choices[0].message.content)

# completion = client.chat.completions.create(
#     model="Qwen/Qwen2.5-VL-72B-Instruct",
#     messages=[
#         {
#             "role": "user",
#             "content": [
#                 {"type": "text", "text": "Describe the images in one sentence."},
#                 {
#                     "type": "image_url",
#                     "image_url": {
#                         "url": "https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg"
#                     },
#                 },
#                 {
#                     "type": "image_url",
#                     "image_url": {
#                         "url": "https://upload.wikimedia.org/wikipedia/commons/3/3d/Nicu%C8%99or_Dan_%288_mai_2025%29.jpg"
#                     },
#                 },
#             ],
#         }
#     ],
# )

# print(completion.choices[0].message)


The critical illnesses covered under Optima Restore include Cancer, Open Chest CABG, First Heart Attack, Kidney Failure, Major Organ/Bone Marrow Transplant, Multiple Sclerosis, Permanent Paralysis of Limbs, and Stroke.


In [43]:
from smolagents import Tool

class SaveContent(Tool):
    name = "SaveContent"
    description = "Saves content to a file."
    inputs = {
        "content": {
            "type": "string",
            "description": "The content to save."
        },
        "filename": {
            "type": "string",
            "description": "The filename."
        }
    }
    output_type = "string"

    def forward(self, content: str, filename: str ):
        with open(filename, "w") as f:
            f.write(content)
        return f"Content saved to {filename}"



In [ ]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), 
                         PythonInterpreterTool(), WikipediaSearchTool(), FinalAnswerTool()], 
                  model=InferenceClientModel(bill_to="VitalNest"),
                  additional_authorized_imports=["os", "requests", "bs4"],
                 max_steps=10, planning_interval=2,
                 )

In [ ]:
agent.run(f"""Search for multi latent attention in wikipedia.
If you cannot find a wikipedia page for it, search the entire wikipedia site for it.
          Retrieve the top search results and the relevant content.""")

In [55]:
from smolagents import ToolCallingAgent

In [87]:
tool_calling_agent = ToolCallingAgent(model=InferenceClientModel(model_id="Qwen/Qwen3-30B-A3B", bill_to="VitalNest"),
                 tools=[DuckDuckGoSearchTool()], verbosity_level=0)

In [ ]:
ans = tool_calling_agent.run("""Search for `topic`:multi latent attention in wikipedia.
If you cannot find a wikipedia page for it, search the entire wikipedia site for it.
Look at the search results, if none of them contain information about the topic, 
                             then perform a normal search and return a summary of the retrieved content
                             along with the search result links.
"""
)

In [ ]:
ans

"No Wikipedia page found for 'multi latent attention.' However, 'Multi-Head Latent Attention (MLA)' is a related concept discussed in articles like 'Attention Evolved' and 'TransMLA,' focusing on optimizing LLMs with low-rank compression."

In [ ]:
tool_calling_agent.provide_final_answer("""Search for multi latent attention in wikipedia.
If you cannot find a wikipedia page for it, search the entire wikipedia site for it.
          Retrieve the top search results and the relevant content.
                       Return only the final answer.""")

'<think>\nOkay, the user wants me to search for "multi latent attention" on Wikipedia. Let me start by checking if there\'s a direct page for that term. I\'ll try the standard Wikipedia search first.\n\nHmm, when I search "multi latent attention" on Wikipedia, there\'s no specific page dedicated to that exact term. The results might be empty or not relevant. Since the user mentioned that if there\'s no page, I should search the entire Wikipedia site. But even after that, the top results from the web search (like the Medium articles, arXiv papers, and tech blogs) don\'t point to a Wikipedia page. \n\nLooking at the sources provided, most of them are from external websites, not Wikipedia. The user\'s task was to check Wikipedia first, then the entire site if needed. But since the web search results don\'t include Wikipedia pages, it\'s likely that there\'s no official page on Wikipedia about "multi latent attention." \n\nThe user also wants the top search results and relevant content. Th

In [1]:
import torch
from PIL import Image
from transformers.utils.import_utils import is_flash_attn_2_available

from colpali_engine.models import ColQwen2_5, ColQwen2_5_Processor


In [ ]:

model = ColQwen2_5.from_pretrained(
    pretrained_model_name_or_path="vidore/colqwen2.5-v0.2",
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",  # or "mps" if on Apple Silicon
    attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
).eval()
processor = ColQwen2_5_Processor.from_pretrained("vidore/colqwen2.5-v0.2", use_fast=True)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:

# Your inputs
images = [
    Image.new("RGB", (128, 128), color="white"),
    Image.new("RGB", (128, 128), color="black"),
]
queries = [
    "Is attention really all you need?",
    "What is the amount of bananas farmed in Salvador?",
]

# Process the inputs
batch_images = processor.process_images(images).to(model.device)
batch_queries = processor.process_queries(queries).to(model.device)

# Forward pass
with torch.no_grad():
    image_embeddings = model(**batch_images)
    query_embeddings = model(**batch_queries)

scores = processor.score_multi_vector(query_embeddings, image_embeddings)
